In [8]:
# Import the functions used in this project
# from private_pkg.functions import *

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [ ]:
%%writefile private_pkg/functions.py

import numpy as np
import pandas as pd
import requests # to send slack msg
import json # to send slack msg
from datetime import datetime
from IPython.display import display, Markdown

def getNullDataInfo(df):
    """
        패러미터 종류 : df
        df : 컬럼별로 np.nan데이터의 총 개수를 DataFrame형식으로 반환한다.
    """
    df = df.isnull().sum().reset_index()
    df.columns = ["Feature_name", "Sum of null data"]
    return df

def getNullDataDetailInfo(df, specify_col = [], see_list_detail = False, visit_number_threshold = 2):
    """
        패러미터 종류 : df, specify_col, see_list_detail
        df : 확인하고자하는 데이터프레임을 넣어준다.
        specify_col : 보고싶은 컬럼들의 이름을 리스트형식으로 넣어준다. default는 []이며, 이 경우에는 모든 컬럼에대한 정보를 다 보여준다. 
        see_list_detail : 분포를 리스트형식 파일로도 보고싶은 경우에 True를 넣어준다. default는 False
        visit_number_threshold : 0보다 큰 정수를 넣어줘야한다. default는 2
    """
    df_ = df[df["Upc"].isnull()]
    if specify_col == []:
        cols = list(df_.columns)
    else:
        cols = specify_col
    for col in cols:
        unique = df_[col].unique()
        print()
        print(str(col) + " 컬럼의 유니크 데이터 정보(" + str(len(unique)) + "종류)")
        if see_list_detail:
            print(unique)
        print()
        print("데이터별 분포")
        __getNullDataDistributionByColumn(df_, col, see_list_detail, visit_number_threshold, df)        
        print()
        if len(cols) > 1:
            print("================================================================================")
            
def displayDataByVisitNumber(df, vn):
    display(df[df["VisitNumber"] == vn])

def makeDfWeWanted(df, df_train, is_use_negative_scan_count = True):
    """
        패러미터 정보 : 
        df에는 변환하고자하는 df를 넣어준다.
        df_train에는 train데이터 프레임을 반드시 넣어준다.
        is_use_negative_scan_count는 True가 default이다.
    """
    cols = df.columns
    df_ = df.copy()
    df_["DepartmentDescription"] = df_["DepartmentDescription"].apply(lambda a : "Null" if type(a) is float else a)
    dd_cols = [uq if not type(uq) is float else "Null" for uq in df_train["DepartmentDescription"].unique()]
    dd_cols.insert(0, "VisitNumber")
    dd_cols.append("TripType")
    result_df = pd.DataFrame(columns = dd_cols)
    dict_ = dict(zip(dd_cols, np.zeros(len(dd_cols)).astype(int)))
    if "TripType" in cols:
        df_ = pd.DataFrame(df_.groupby(["VisitNumber", "TripType", "DepartmentDescription"]).sum()["ScanCount"]).reset_index()
    else:
        df_ = pd.DataFrame(df_.groupby(["VisitNumber", "DepartmentDescription"]).sum()["ScanCount"]).reset_index()
    return __makeDf(df_, is_use_negative_scan_count)

def __makeDf(df, is_use_negative_scan_count):
    display(Markdown("##### makeDfWeWanted함수는 !! 데이터프레임을 반환하는 \
    함수이므로 변수로 받아주셔야합니다!! 오래 걸리는 작업(맥북 프로 기준 3분)이므로 지금 실수하셨다면 빨리 커널 종료하고 다시 시도해주세요."))
    vn_uq_li = df["VisitNumber"].unique()
    li = []
    scan_count_min = 0
    if is_use_negative_scan_count:
        scan_count_min = -df["ScanCount"].min()
    for i, vn in enumerate(vn_uq_li):
        tmp_df = df[df["VisitNumber"] == vn]
        tripType = tmp_df["TripType"].unique()[0]
        space = np.zeros(71).astype(int) + scan_count_min
        space[0] = vn
        space[len(space) - 1] = tripType
        for row_nbr in tmp_df.index:
            dd = tmp_df.loc[row_nbr]["DepartmentDescription"]
            scan_cnt = tmp_df.loc[row_nbr]["ScanCount"] + scan_count_min
            idx = dd_cols.index(dd)
            if is_use_negative_scan_count:
                space[idx] = scan_cnt
            else:
                if scan_cnt > 0:
                    space[idx] = scan_cnt
        li.append(space)
        if (i % 5000) == 0:    
            print(str(i) + "명 진행됨. 아직 " + str(len(vn_uq_li) - i) + "명 데이터 남음.")
    return pd.DataFrame(li, columns=dd_cols) 
    
def __getNullDataDistributionByColumn(df, col, see_list_detail, visit_number_threshold, original_df):
    try:
        unique_li = sorted(df[col].unique())
    except:
        unique_li = df[col].unique()
    data_name_li = []
    len_li = []
    if col == "VisitNumber":
        data_name_li, len_li = __getMoreAnalyzedVisitNumber(df, unique_li, col, visit_number_threshold, original_df)
    else:
        for unique in list(unique_li):
            data_name_li.append(unique)
            if type(unique) is not str:
                unique = float(unique)
            if type(unique) is float:
                if np.isnan(unique):
                    len_li.append(len(df[df[col].isnull()]))
                else:
                    len_li.append(len(df[df[col] == unique]))                
            else:        
                len_li.append(len(df[df[col] == unique]))
    if see_list_detail:
        print(dict(zip(data_name_li, len_li)))
    else:
        if len(unique_li) == 1:
            print(dict(zip(data_name_li, len_li)))
    if len(unique_li) > 1 and col != "VisitNumber":
        data_name_li = [ 'Null' if type(d) is float and np.isnan(d) else d for d in data_name_li]
        plt.figure(figsize=(10, 5))
        sns.barplot(x = data_name_li, y = len_li)
        plt.xticks(rotation = 90)
        plt.show()

def __getMoreAnalyzedVisitNumber(df, unique_li, col, visit_number_threshold, original_df):
    df_null_mask = []
    df_pharmacy_mask = []
    data_name_li = []
    len_li = []
    for unique in list(unique_li):
        length = len(df[df[col] == unique])
        if length >= visit_number_threshold:            
            data_name_li.append(unique)
            len_li.append(length)
    
    display(Markdown("## 여기서 세가지 모두 Null인 아이템이란 Upc, DepartmentDescription, FinelineNumber가 다 NaN인 경우를 말한다.\
    또한 두가지가 Null인 아이템이란 Upc, FinelineNumber가 NaN이면서 DepartmentDescription은 PHARMACY RX인 경우를 말한다."))
    
    df_dd_null = df[df["DepartmentDescription"].isnull()]
    df_null_mask = df_dd_null["VisitNumber"].isin(data_name_li)
    df_dd_null_visit_number_above_threshold = df_dd_null[df_null_mask]
    ppl_nbr = len(df_dd_null_visit_number_above_threshold["VisitNumber"].unique())
    print("세가지 컬럼이 모두 Null인 " + str(visit_number_threshold) + "가지 이상 종류의 아이템을 산 사람은 총 " + str(ppl_nbr) + "명이다.")
    __drawTripTypeBarPlot(df_dd_null_visit_number_above_threshold, original_df, df, False, visit_number_threshold)
    
    df_dd_pharmacy = df[df["DepartmentDescription"] == "PHARMACY RX"]
    df_pharmacy_mask = df_dd_pharmacy["VisitNumber"].isin(data_name_li)
    df_dd_pharmacy_visit_number_above_threshold = df_dd_pharmacy[df_pharmacy_mask]
    ppl_nbr = len(df_dd_pharmacy_visit_number_above_threshold["VisitNumber"].unique())
    print("두가지 컬럼이 모두 Null인 " + str(visit_number_threshold) + "가지 이상 종류의 아이템을 산 사람은 총 " + str(ppl_nbr) + "명이다.")
    __drawTripTypeBarPlot(df_dd_pharmacy_visit_number_above_threshold, original_df, df, True, visit_number_threshold)
    
    print("위 두가지 경우의 합이 VisitNumber컬럼의 유니크한 데이터 총 개수와 다른 이유는 23757, 50745")
    print("VisitNumber에 해당하는 두 명이 세가지 다 Null인 아이템과 Pharmacy RX 제품 두개다 구매했기 때문이다.")
    display(df[df["VisitNumber"] == 23757])
    display(df[df["VisitNumber"] == 50745])
    
    __getDetailInfoAboutVisitNumberBoughtNotNullProduct(data_name_li, original_df, visit_number_threshold)
    
    return data_name_li, len_li

def __drawTripTypeBarPlot(df, original_df, original_df_with_only_null, isNull, visit_number_threshold):
    col = "TripType"
    unique_li = sorted(original_df[col].unique())
    data_name_li = []
    len_li = []
    for unique in list(unique_li):
        length = len(df[df[col] == unique])         
        data_name_li.append(unique)
        len_li.append(length)
    plt.figure(figsize=(10, 5))
    sns.barplot(x = data_name_li, y = len_li)
    plt.xticks(rotation = 90)
    plt.show()
    if isNull:
        percentage = str(round(np.array(len_li).max()/sum(len_li), 2) * 100) + "%"
        display(Markdown('## **Pharmacy rx인 경우에는 {}가 TripType 5에 해당한다.**'.format(percentage)))
        display(Markdown('## **두가지 컬럼이 Null인 {}가지 이상 종류의 아이템을 산사람들의 분포이다.**'.format(visit_number_threshold)))        
    else:
        percentage = str(round(np.array(len_li).max()/sum(len_li), 2) * 100) + "%"
        display(Markdown('## **Upc, DepartmentDescription, FinelineNumber 모두 Null인 경우에는 999 type이 {}를 차지한다.**'.format(percentage)))        
        display(Markdown('## **세가지 컬럼이 모두 Null인 {}가지 이상 종류의 아이템을 산사람들의 분포이다.**'.format(visit_number_threshold)))        
        
def __getDetailInfoAboutVisitNumberBoughtNotNullProduct(data_name_li, original_df_, visit_number_threshold):
    col = "VisitNumber"
    original_df = original_df_.copy()
    cut_off_df = original_df_[original_df_["Upc"].isnull()]
    df_null_mask = original_df["VisitNumber"].isin(data_name_li)
    null_df = original_df[df_null_mask]
    null_li = null_df[col].unique()
    uq_li = []
    uq_li_who_bought_only_null_products = []
    for li in null_li:
        if len(null_df[null_df[col] == li]["Upc"].unique()) > 1:
            uq_li.append(li)
        else:
            uq_li_who_bought_only_null_products.append(li)
            
    null_df = original_df_[original_df_[col].isin(uq_li)]    
    cut_off_null_df = cut_off_df[cut_off_df[col].isin(uq_li)]

    display(Markdown("#### Null이 포함된 " + str(visit_number_threshold) + "가지 이상의 아이템을 산 사람들의 null 아이템 정보(df.tail)"))
    display(cut_off_null_df.tail())     
    display(Markdown("#### Null이 포함된 " + str(visit_number_threshold) + "가지 이상의 아이템을 산 사람들의 모든 아이템 정보(df.tail)"))
    display(null_df.tail())     
#     display(null_df.groupby("VisitNumber"))
    
    display(Markdown("## 총 " + __setColoredText(len(null_li)) + "명의 사람들 중에 " + __setColoredText(len(uq_li)) + "(" + __setColoredText(round(len(uq_li)/len(null_li), 2) * 100) + "%)명에 해당하는 사람들은 Null 데이터가 아닌 아이템도 샀다."))
    display(Markdown("## 즉, " + __setColoredText(len(null_li) - len(uq_li)) + "명의 사람만이 Null 데이터만 가진 아이템을 샀다."))
    
#     display(len(uq_li_who_bought_only_null_products), uq_li_who_bought_only_null_products[:5])
    df_only_bought_null_products = cut_off_df[cut_off_df["VisitNumber"].isin(uq_li_who_bought_only_null_products)]
    
    df_only_bought_null_pharmacy_products = df_only_bought_null_products[df_only_bought_null_products["DepartmentDescription"] == "PHARMACY RX"]
    df_only_bought_null_not_pharmacy_products = df_only_bought_null_products[df_only_bought_null_products["DepartmentDescription"] != "PHARMACY RX"]
    
    display(df_only_bought_null_pharmacy_products.tail())
    unique_nbr = len(df_only_bought_null_pharmacy_products["VisitNumber"].unique())
    display(Markdown("## 그 " +  str(len(null_li) - len(uq_li)) + "명 중에 " + str(unique_nbr) + " 명이 Pharmacy 제품만을 산사람이다."))
    
    display(df_only_bought_null_not_pharmacy_products.tail())
    uq_li_999 = df_only_bought_null_not_pharmacy_products["VisitNumber"].unique()
    unique_nbr_ = len(uq_li_999)
    display(Markdown("## 그 " +  str(len(null_li) - len(uq_li)) + "명 중에 " + str(unique_nbr_) + " 명이 Null 제품만을 산사람이다."))
#     display(df_only_bought_null_not_pharmacy_products["TripType"].unique())
    display(Markdown("## 이 사람들의 TripType은 모두 "+ __setColoredText(999) + "이다."))
    
    special_vn_li = []
    for nbr in uq_li_999:
        sumOfScanCount = df_only_bought_null_not_pharmacy_products[df_only_bought_null_not_pharmacy_products["VisitNumber"] == nbr]["ScanCount"].sum()
        if sumOfScanCount > 0:
            special_vn_li.append(nbr)
    display(Markdown("## 그 중에서 VisitNumber별로 ScanCount의 Sum이 0이상인 경우(실제로 NaN아이템을 산사람)의 VisitNumber list다."))            
    print(special_vn_li)
    
def __setColoredText(txt, color_rgb = "red"):
    return "<span style='color:" + color_rgb +"'>" + str(txt) + "</span>"

def saveDataFrameToCsv(df, fileName, idx = False):
    """
        넘겨준 df를 filename + 년월일시간분 의 format으로 이루어진 이름의 파일로 생성해준다.
        index를 True로 넘겨주면 저장할 때 아규먼트로 index=True를 넣어주게 된다.
    """
    fileName += "_" + datetime.now().strftime("%Y%m%d%H%M") + ".csv"
    return df.to_csv(fileName, index = idx)

def sendSlackDm(url, text):
    """
        Parameter :
            각자 받은 url을 넣어준다.
            text에는 보낼 글 내용
    """
    webhook_url = url
    slack_data = {'text': text}
    response = requests.post(
        webhook_url,
        data=json.dumps(slack_data),
        headers={'Content-Type': 'application/json'}
    )
    if response.status_code != 200:
        raise ValueError(
            'Request to slack returned an error %s, the response is:\n%s'%(response.status_code, response.text)
    )

In [10]:
import pickle as pkl
slack_url = pkl.load(open("send_url.pickle", "rb"))

In [11]:
df = makeDfWeWanted(df_train, df_train)

In [4]:
df_train[df_train["DepartmentDescription"] == "HEALTH AND BEAUTY AIDS"]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
552573,36,164059,Wednesday,3.607344e+11,1,HEALTH AND BEAUTY AIDS,2250.0
612184,999,182398,Saturday,3.099723e+10,-1,HEALTH AND BEAUTY AIDS,3501.0


## HEALTH AND BEAUTY AIDS 는 test에는 없는 DepartmentDescription이다.

In [12]:
dd_cols = [uq if not type(uq) is float else "Null" for uq in df_train["DepartmentDescription"].unique()]

In [13]:
dd_cols.insert(0, "VisitNumber")

In [14]:
dd_cols.append("TripType")

In [15]:
df_wewanted = pd.DataFrame(columns = dd_cols)

In [16]:
vn_uq_li = df["VisitNumber"].unique()

In [17]:
len(df_wewanted)

0

In [25]:
len(vn_uq_li) % 95674

0

In [26]:
df_wewanted = pd.DataFrame(columns = dd_cols)

In [80]:
dict_ = dict(zip(cols, [li]))

In [ ]:
pd.DataFrame

In [133]:
df["ScanCount"].min()

-17

In [151]:
def makeDf_1(is_use_negative_scan_count = True):
    vn_uq_li = df["VisitNumber"].unique()
    scan_count_min = 0
    if is_use_negative_scan_count:
        scan_count_min = -df["ScanCount"].min()
    li = [np.zeros(71).astype(int) - scan_count_min for l in range(len(vn_uq_li))]        
    result_df = pd.DataFrame(li, columns=dd_cols)
    for i, vn in enumerate(vn_uq_li):
        tmp_df = df[df["VisitNumber"] == vn]
        tripType = tmp_df["TripType"].unique()[0]
        space = np.zeros(71).astype(int) + scan_count_min
        space[0] = vn
        space[len(space) - 1] = tripType
        for row_nbr in tmp_df.index:
            dd = tmp_df.loc[row_nbr]["DepartmentDescription"]
            scan_cnt = tmp_df.loc[row_nbr]["ScanCount"] + scan_count_min
            idx = dd_cols.index(dd)
            if is_use_negative_scan_count:
                space[idx] = scan_cnt
            else:
                if scan_cnt > 0:
                    space[idx] = scan_cnt
        result_df.loc[i] = space
        if (i % 5000) == 0:    
            print(str(i) + "명 진행됨. 아직 " + str(len(vn_uq_li) - i) + "명 데이터 남음.")
    return result_df

In [152]:
def makeDf_2(is_use_negative_scan_count = True):
    vn_uq_li = df["VisitNumber"].unique()
    li = []
    scan_count_min = 0
    if is_use_negative_scan_count:
        scan_count_min = -df["ScanCount"].min()
    for i, vn in enumerate(vn_uq_li):
        tmp_df = df[df["VisitNumber"] == vn]
        tripType = tmp_df["TripType"].unique()[0]
        space = np.zeros(71).astype(int) + scan_count_min
        space[0] = vn
        space[len(space) - 1] = tripType
        for row_nbr in tmp_df.index:
            dd = tmp_df.loc[row_nbr]["DepartmentDescription"]
            scan_cnt = tmp_df.loc[row_nbr]["ScanCount"] + scan_count_min
            idx = dd_cols.index(dd)
            if is_use_negative_scan_count:
                space[idx] = scan_cnt
            else:
                if scan_cnt > 0:
                    space[idx] = scan_cnt
        li.append(space)
        if (i % 5000) == 0:    
            print(str(i) + "명 진행됨. 아직 " + str(len(vn_uq_li) - i) + "명 데이터 남음.")
    return pd.DataFrame(li, columns=dd_cols)

In [35]:
saveDataFrameToCsv(df_wewanted, "df_we_wanted")

In [38]:
from sklearn.naive_bayes import MultinomialNB
# model_mult = MultinomialNB().fit(X, y)

In [42]:
cols = [uq if not type(uq) is float else "Null" for uq in df_train["DepartmentDescription"].unique()]

In [121]:
X_1 = df_1[:-1][cols]

In [132]:
X_1.loc[13789]

FINANCIAL SERVICES              0
SHOES                           0
PERSONAL CARE                  11
PAINT AND ACCESSORIES           0
DSD GROCERY                     0
MEAT - FRESH & FROZEN           0
DAIRY                           0
PETS AND SUPPLIES               0
HOUSEHOLD CHEMICALS/SUPP       10
Null                            0
IMPULSE MERCHANDISE            10
PRODUCE                         0
CANDY, TOBACCO, COOKIES         0
GROCERY DRY GOODS               2
BOYS WEAR                       0
FABRICS AND CRAFTS              0
JEWELRY AND SUNGLASSES          0
MENS WEAR                       0
ACCESSORIES                    11
HOME MANAGEMENT                 0
FROZEN FOODS                    0
SERVICE DELI                    0
INFANT CONSUMABLE HARDLINES     0
PRE PACKED DELI                 0
COOK AND DINE                  -2
PHARMACY OTC                   -5
LADIESWEAR                     11
COMM BREAD                      0
BAKERY                          0
HOUSEHOLD PAPE

In [131]:
for col in cols:
    display(X_1[X_1[col] < 0])

,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


...

,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


,FINANCIAL SERVICES,SHOES,PERSONAL CARE,PAINT AND ACCESSORIES,DSD GROCERY,MEAT - FRESH & FROZEN,DAIRY,PETS AND SUPPLIES,HOUSEHOLD CHEMICALS/SUPP,Null,...,SWIMWEAR/OUTERWEAR,OTHER DEPARTMENTS,MEDIA AND GAMING,FURNITURE,OPTICAL - LENSES,SEASONAL,LARGE HOUSEHOLD GOODS,1-HR PHOTO,CONCEPT STORES,HEALTH AND BEAUTY AIDS


In [43]:
X = df_wewanted[cols]

In [51]:
y = list(df_wewanted["TripType"].values)

In [55]:
model_mult = MultinomialNB().fit(X, y)
sendSlackDm(slack_url, "Done")

In [56]:
model_mult.classes_

array([  3,   4,   5,   6,   7,   8,   9,  12,  14,  15,  18,  19,  20,
        21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,
        34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44, 999])

In [58]:
pred_y = model_mult.predict(X)

In [61]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [64]:
np.trace(confusion_matrix(y, pred_y))/len(X)

0.4660513828208291

In [62]:
print(classification_report(y, pred_y))

             precision    recall  f1-score   support

          3       0.80      0.87      0.84      3643
          4       0.07      0.04      0.05       346
          5       0.75      0.81      0.78      4593
          6       0.48      0.55      0.51      1277
          7       0.60      0.66      0.63      5752
          8       0.36      0.52      0.42     12161
          9       0.57      0.45      0.50      9464
         12       0.15      0.21      0.18       269
         14       0.05      0.50      0.09         4
         15       0.37      0.49      0.42       978
         18       0.38      0.53      0.44       549
         19       0.39      0.23      0.29       375
         20       0.47      0.73      0.57       637
         21       0.41      0.76      0.53       641
         22       0.45      0.22      0.30       928
         23       0.27      0.20      0.23       139
         24       0.51      0.64      0.57      2609
         25       0.56      0.74      0.64   

In [95]:
df.loc[0]["DepartmentDescription"]

'FINANCIAL SERVICES'

In [105]:
df[df["VisitNumber"]==7]["TripType"]

30

In [83]:
for loc in range(len(df[:5])):
    display(df.loc[loc])

VisitNumber                               5
TripType                                999
DepartmentDescription    FINANCIAL SERVICES
ScanCount                                -1
Name: 0, dtype: object

VisitNumber                          7
TripType                            30
DepartmentDescription    PERSONAL CARE
ScanCount                            1
Name: 1, dtype: object

VisitNumber                  7
TripType                    30
DepartmentDescription    SHOES
ScanCount                    1
Name: 2, dtype: object

VisitNumber                  8
TripType                    26
DepartmentDescription    DAIRY
ScanCount                    2
Name: 3, dtype: object

VisitNumber                        8
TripType                          26
DepartmentDescription    DSD GROCERY
ScanCount                          1
Name: 4, dtype: object

In [68]:
df_ = df_train.copy()

In [69]:
df_["DepartmentDescription"] = df_["DepartmentDescription"].apply(lambda a : "Null" if type(a) is float else a)

In [70]:
df__ = pd.DataFrame(df_.groupby(["VisitNumber", "TripType", "DepartmentDescription"]).sum()["ScanCount"]).reset_index()

In [71]:
len(df__["VisitNumber"].unique())

95674